In [1]:
import numpy as np
import time
import copy
import random
import Ipynb_importer
from RBM import *
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import tensorflow as tf

importing Jupyter notebook from RBM.ipynb


/home/zda/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/zda/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/zda/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/zda/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/zda/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 f

In [3]:
class SDNE(nn.Module):
    def __init__(self,config):
        super(SDNE, self).__init__()
        self.is_variables_init = False
        self.config=config  #config =dict
        
        ######define variables ###
        self.layers=len(config.struct)  #struct:[-1(1),100....]
        self.struct=config.struct
        self.sparse_dot=config.sparse_dot   #config.sparse_dot 不存在啊
        struct=self.struct
        
        ###
        self.WE1=Parameter(torch.FloatTensor(np.random.normal(size=(struct[0],struct[1]))))
        self.be1=Parameter(torch.FloatTensor(np.random.normal(size=(struct[1]))))
        
        self.WD1=Parameter(torch.from_numpy(np.random.normal(size=(struct[1],struct[0]))))
        self.bd1=Parameter(torch.from_numpy(np.random.normal(size=(struct[0]))))
        
#         self.W={}
#         self.b={}
#         for i in range(self.layers-1):
#             name="encoder"+str(i)
#             self.W[name]=Parameter(torch.from_numpy(np.random.normal(size=(struct[i],struct[i+1]))))
#             self.b[name]=Parameter(torch.zeros(struct[i+1]))
#         struct.reverse()
#         for i in range(self.layers-1):
#             name="decoder"+str(i)
#             self.W[name]=Parameter(torch.from_numpy(np.random.normal(size=(struct[i],struct[i+1]))))
#             self.b[name]=Parameter(torch.zeros(struct[i+1]))
#         struct.reverse()
        ###########################
        
        ############define input###
    def forward(self,data,adj,config):
        self.X=data
#         self.do_variables_init(data)
        self._make_compute_graph()
        self.loss=self._make_loss(config,adj)
        return self.loss
        
        
    def _make_compute_graph(self):
        def encoder(X):
            for i in range(self.layers-1):
                name="encoder"+str(i)
                X=F.sigmoid(torch.matmul(X,self.WE1+self.be1))
            return X

        def encoder_sp(X):
            for i in range(self.layers-1):
                name="encoder"+str(i)
                if i ==0:
                    X=F.sigmoid(torch.sparse.mm(X,self.WE1)+self.be1)
                else:
                    X=F.sigmoid(torch.matmul(X,self.WE1)+self.be1)
            return X

        def decoder(X):
            for i in range(self.layers-1):
                name="decoder"+str(i)
                X=F.sigmoid(torch.matmul(X,self.WD1)+self.bd1)
            return X

        if self.sparse_dot:
            self.H=encoder_sp(self.X_sp)
        else:
            self.H=encoder(self.X)

        self.X_reconstruct=decoder(self.H)

####
#     def _make_compute_graph(self):
#         def encoder(X):
#             for i in range(self.layers-1):
#                 name="encoder"+str(i)
#                 X=F.sigmoid(torch.matmul(X,self.W[name]+self.b[name]))
#             return X

#         def encoder_sp(X):
#             for i in range(self.layers-1):
#                 name="encoder"+str(i)
#                 if i ==0:
#                     X=F.sigmoid(torch.sparse.mm(X,self.W[name])+self.b[name])
#                 else:
#                     X=F.sigmoid(torch.matmul(X,self.W[name])+self.b[name])
#             return X

#         def decoder(X):
#             for i in range(self.layers-1):
#                 name="decoder"+str(i)
#                 X=F.sigmoid(torch.matmul(X,self.W[name])+self.b[name])
#             return X

#         if self.sparse_dot:
#             self.H=encoder_sp(self.X_sp)
#         else:
#             self.H=encoder(self.X)

#         self.X_reconstruct=decoder(self.H)

    ####
    def _make_loss(self,config,adj_mini_batch):
        self.adjacent_matriX=adj_mini_batch
        def get_1st_loss_link_sample(self, Y1, Y2):
            return torch.sum(torch.pow(Y1 - Y2, 2))
        def get_1st_loss(H, adj_mini_batch):
            D = torch.diag(torch.sum(adj_mini_batch,1))
            L = D - adj_mini_batch ## L is laplation-matriX
            return 2*torch.trace(torch.matmul(torch.matmul(torch.transpose(H,0,1),L),H))


        def get_2nd_loss(X, newX, beta):
            B = X * (beta - 1) + 1
            return torch.sum(torch.pow((newX - X)* B, 2))

#             def get_reg_loss(weight, biases):
#                 ret = torch.sum([sum(w**2)/() for w in weight.values()])
#                 ret = ret + tf.add_n([tf.nn.l1_loss(b) for b in biases.values()])
#                 return ret
        self.loss_2nd = get_2nd_loss(self.X, self.X_reconstruct, config.beta)
        self.loss_1st = get_1st_loss(self.H, self.adjacent_matriX)
#         self.loss_xxx = tf.reduce_sum(tf.pow(self.X_reconstruct,2)) 
    # we don't need the regularizer term, since we have nagetive sampling.
#         self.loss_reg = get_reg_loss(self.W, self.b) 
        return config.gamma * self.loss_1st + config.alpha * self.loss_2nd #+ config.reg * self.loss_reg

        
    def do_variables_init(self,data):
        shape=self.struct
        myRBMs=[]
        for i in range(len(shape)-1):
            myRBM=rbm([shape[i],shape[i+1]],{"batch_size": self.config.dbn_batch_size,
                                             "learning_rate":self.config.dbn_learning_rate})
            myRBMs.append(myRBM)
            for epoch in range(self.config.dbn_epochs):
                error=0
                for batch in range(0,data[0].N,self.config.dbn_batch_size):
                    mini_batch=data[0].sample(self.config.dbn_batch_size).X
                    for k in range(len(myRBMs)-1):
                        mini_batch=myRBMs[k].getH(mini_batch)
                    error+=myRBM(mini_batch)
                print("rbm epochs:", epoch, "error : ", error)
                W, bv, bh = myRBM.W,myRBM.bv,myRBM.bh
                name = "encoder" + str(i)
                self.WE1.data= W
                self.be1.data=bh
                name = "decoder" + str(self.layers - i - 2)
                self.WD1.data=W.transpose(0,1)
                self.bd1.data=bv
            self.is_Init=True
#     def get_embedding(self,data):
#         return self.H